# Stacking with TimeSeries Split

### Create metafeatures for the train set

In [ ]:
# dates for the second level
dates_level2 = [28, 29, 30, 31, 32, 33]

# train dates
dates_train = X_train.date_block_num
# dates_train_level2 = dates_train[dates_train.isin(dates)]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train[dates_train.isin(dates_level2)]

# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], len(best_params)+1])
X_train_level2[:, len(best_params)] = y_train_level2

meta_index_begin = 0
meta_index_end = 0

# Now fill `X_train_level2` with metafeatures
for cur_block_num in tqdm_notebook(dates_level2):
    
    # split data
    train_index = X_train.loc[dates_train <  cur_block_num].index
    test_index  = X_train.loc[dates_train == cur_block_num].index
    
    X_train_l2 = X_train.loc[train_index, :]
    X_test_l2 =  X_train.loc[test_index, :]

    y_train_l2 = y_train[train_index]
    y_test_l2 =  y_train[test_index]
    
    meta_index_end += y_test_l2.shape[0]
    
    # predict metafeatures for each of LGBM regressors
    for i, params in enumerate(tqdm_notebook(best_params)):
        lgb = LGBMRegressor(**params)
        reg = lgb.fit(X_train_l2, y_train_l2)
        pred = lgb.predict(X_test_l2)
        X_train_level2[meta_index_begin:meta_index_end, i] = pred

        del lgb, reg, pred
        gc.collect()
        
    meta_index_begin = meta_index_end
        
X_train_level2 = pd.DataFrame(X_train_level2, columns=[f'lgbr_{i+1}' for i in range(len(best_params))]+['gt'])
X_train_level2.to_pickle('LGBM_X_train_level2.pkl')

### Create metafeatures for the test set

In [ ]:
X_test_level2 = np.zeros([X_test.shape[0], len(best_params)])

for i, params in enumerate(tqdm_notebook(best_params)):
    lgb = LGBMRegressor(**params)
    reg = lgb.fit(X_train, y_train)
    pred = lgb.predict(X_test)
    X_test_level2[:, i] = pred
    
    del lgb, reg, pred
    gc.collect()
    
X_test_level2 = pd.DataFrame(X_test_level2, columns=[f'lgbr_{i+1}' for i in range(len(best_params))])
X_test_level2.to_pickle('LGBM_X_test_level2.pkl')

### Stacking with Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

X = X_train_level2[[c for c in X_train_level2.columns if c != 'gt']]
y = X_train_level2['gt']

lr = LinearRegression()
lr.fit(X, y)

pred = lr.predict(X_test_level2)